In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

from tqdm.keras import TqdmCallback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import datetime

In [5]:
# Read interaction matrix pickle file
size = 'demo'
fillna_value = '0'
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix_df = pd.read_pickle(interaction_matrix_file_path)
print('Interaction matrix df shape:                      ',interaction_matrix_df.shape)

user_matrix_df_file_path = f'./files/pickle/user_matrix_{size}_{fillna_value}.pkl'
article_matrix_df_file_path = f'./files/pickle/article_matrix_{size}_{fillna_value}.pkl'

user_matrix_df = pd.read_pickle(user_matrix_df_file_path)
article_matrix_df = pd.read_pickle(article_matrix_df_file_path)
print('User embedding df shape:                         ',user_matrix_df.shape)
print('Article embedding df shape:                      ',article_matrix_df.shape)

Interaction matrix df shape:                       (1590, 4247)
User embedding df shape:                          (1590, 300)
Article embedding df shape:                       (4247, 300)


In [22]:
interaction_matrix_df.head(2)

article_id_fixed,9251369,9730301,9733713,9737535,9738292,9738334,9740021,9740161,9740174,9740356,...,9700074,9737345,6404190,9448400,9728595,9726404,8166777,9582969,9627627,9674356
user_id,,,,,,,,,,,,,,,,,,,,,
11313,299.0,323.0,7.0,1038.0,32.0,4.0,82.0,4.0,674.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13538,0.0,3.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
user_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
user_id,,,,,,,,,,,,,,,,,,,,,
11313,-2.825051,-0.367806,0.030133,-2.053737,3.554389,1.388510,-2.562073,-1.092667,-2.818263,2.857499,...,1.205560,0.040370,-0.489170,-0.565870,0.375311,0.611979,0.917979,-1.692447,-0.107961,0.027109
13538,0.179039,-0.434059,0.162799,-0.843279,-0.408800,-0.539446,-0.002827,-0.149006,-0.203626,-0.120278,...,0.086555,-0.021488,-0.484404,0.346705,0.084665,0.250012,0.218374,-1.116250,-0.028726,0.015211


In [26]:
article_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
9251369,-0.30518,-0.039914,0.000081,-0.219868,0.370485,0.148578,-0.270606,-0.111830,-0.298030,0.296399,...,0.214525,0.005896,0.017027,-0.037189,0.014235,0.017839,0.030130,-0.044657,-0.001425,0.000168
9730301,-0.03106,0.440268,0.364339,0.325598,-0.209704,-0.467719,-0.558553,-1.317881,-0.540366,0.495923,...,-0.542328,-0.056828,-1.397505,0.700664,0.265783,0.491832,0.789057,-1.740790,-0.220962,0.057037


In [3]:
# Convert the dataframes to numpy arrays
user_vectors = user_matrix_df.values
article_vectors = article_matrix_df.values
interaction_matrix = interaction_matrix_df.values

In [20]:
interaction_matrix

array([[299., 323.,   7., ...,   0.,   0.,   0.],
       [  0.,   3.,  14., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  80.,   0., 202.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [21]:
user_vectors

array([[-2.1178677 , -0.27138555,  0.02235232, ...,  0.06705233,
         0.00690998, -0.02915382],
       [ 0.14442271, -0.32121077,  0.12210255, ...,  0.26654083,
         0.02558004, -0.03103652],
       [-0.07078145,  0.00949412, -0.06409635, ...,  0.6344097 ,
         0.03215543, -0.03284584],
       ...,
       [-1.0710751 , -1.2240779 , -1.1862124 , ..., -1.2484508 ,
         0.01716145, -0.03013842],
       [ 0.00340351,  1.3302095 , -0.49117735, ...,  0.07649194,
         0.02649878, -0.03268577],
       [ 0.07754252, -0.16524129,  0.15225354, ...,  0.4926087 ,
         0.03145659, -0.03304515]], dtype=float32)

In [27]:
article_vectors

array([[-2.2022754e-01, -2.4799278e-02, -2.4364472e-04, ...,
         6.3754356e-01,  3.2012891e-02, -3.3416893e-02],
       [-1.3796339e-02,  3.3631572e-01,  2.7363923e-01, ...,
         5.0315276e-02, -1.9716121e-02, -2.4418302e-02],
       [ 1.2082847e+00, -1.6344656e+00,  1.2208464e+00, ...,
         1.6711214e-01,  2.8328052e-02, -2.7332233e-02],
       ...,
       [-2.1239035e-02, -2.9761944e-02, -3.4037966e-02, ...,
         6.4496493e-01,  3.2306235e-02, -3.3439241e-02],
       [ 9.5940428e-03,  5.2178106e-03, -3.0481422e-04, ...,
         6.5300441e-01,  3.2348640e-02, -3.3443421e-02],
       [-6.8259455e-02, -8.3106123e-02, -8.5481010e-02, ...,
         6.3270473e-01,  3.2241564e-02, -3.3432860e-02]], dtype=float32)

In [11]:
# Normalize the embeddings (optional, depending on your use case)
scaler = StandardScaler()
user_vectors = scaler.fit_transform(user_vectors)
article_vectors = scaler.transform(article_vectors)

In [12]:
# Normalize the original interaction matrix
interaction_matrix_normalized = MinMaxScaler().fit_transform(interaction_matrix)


In [13]:
# Get the indices of the non-zero entries in the interaction matrix
user_idx, article_idx = np.where(interaction_matrix_normalized != 0)
read_times = interaction_matrix_normalized[user_idx, article_idx]

In [29]:
print(read_times.shape)
read_times

(212069,)


array([1.0000000e+00, 4.1839376e-01, 4.8712594e-03, ..., 4.4809561e-03,
       3.8922157e-02, 6.6666666e-04], dtype=float32)

In [30]:
# Create the input features by concatenating user and article vectors
X = np.hstack((user_vectors[user_idx], article_vectors[article_idx]))
y = read_times


In [32]:
X

array([[-2.1178677 , -0.27138555,  0.02235232, ...,  0.63754356,
         0.03201289, -0.03341689],
       [-2.1178677 , -0.27138555,  0.02235232, ...,  0.05031528,
        -0.01971612, -0.0244183 ],
       [-2.1178677 , -0.27138555,  0.02235232, ...,  0.16711214,
         0.02832805, -0.02733223],
       ...,
       [ 0.07754252, -0.16524129,  0.15225354, ..., -0.72982407,
         0.02148201, -0.02953955],
       [ 0.07754252, -0.16524129,  0.15225354, ...,  0.38707936,
         0.03066952, -0.01034811],
       [ 0.07754252, -0.16524129,  0.15225354, ...,  0.1557221 ,
         0.02943029, -0.03098933]], dtype=float32)

In [31]:
print(X.shape)
print(y.shape)

(212069, 600)
(212069,)


In [10]:
X.shape

(212069, 600)

In [33]:
# Use only the first 100 rows for testing
X = X[:100]
y = y[:100]

In [34]:
print(X.shape)
print(y.shape)

(100, 600)
(100,)


In [35]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(80, 600)
(20, 600)
(80,)
(20,)


In [36]:
# Define the model
model = Sequential([
    tf.keras.Input(shape=(600,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation = 'linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [37]:

# Prepare TensorBoard callback
log_dir = "files/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
num_epochs = 2
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=16,
                    validation_data=(X_test, y_test),
                    callbacks=[tensorboard_callback, TqdmCallback(verbose=1)],
                    verbose=2)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/2


In [ ]:
# Save the trained model
model.save('recommendation_model.h5')


In [ ]:
# Evaluate the model
train_loss, train_mse = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mse = model.evaluate(X_test, y_test, verbose=1)